Modified from https://lectures.quantecon.org/jl/mccall_model.html

In [1]:
function solve_mccall_model(mcm; tol=1e-5, max_iter=2000, U_iv = 1.0, V_iv = ones(length(mcm.w)))
    α, β, σ, c, γ, w = mcm.α, mcm.β, mcm.σ, mcm.c, mcm.γ, mcm.w

    #Calculate the utility at the wages and unemployment
    u = mcm.u(w, σ)
    u_c = mcm.u(c, σ)
    
    #Initial values
    V = V_iv
    V_new = V_iv
    U = U_iv
    U_iv = U_iv
    error = Inf
    iter = 1   

    #Value function iteration
    while error > tol && iter < max_iter
        V_new = u + β * ((1 - α) * V + α * U)
        U_new = u_c + β * (1 - γ) * U + β * γ * dot(max.(U, V), mcm.p)

        #Check the infinity norm
        error = norm([V_new - V; U_new - U], Inf)
        
        #Update the values
        V = V_new
        U = U_new
        iter += 1
    end

    #Find the policy, where V > U  
    wbarindex = searchsortedfirst(V - U, 0.0)
    if(wbarindex > 0)
        wbar = w[wbarindex]
    else
        wbar = Inf
    end
    return V, U, wbar
end

solve_mccall_model (generic function with 1 method)

In [2]:
using Plots, NamedTuples, Distributions
gr()

#Utility function
u(c, σ) = (c.^(1 - σ) - 1) ./ (1 - σ)

#Wage values
n = 20
w = linspace(10, 20, n)

#The distribution to use for probabilities.
dist = BetaBinomial(n - 1, 600, 400)
p = pdf.(dist, support(dist))

#Named tuple to hold all parameters
mcm = @NT( #Can drop @NT with Julia v0.7/v1.0
    u = u,
    α = 0.2,
    β = 0.98,
    γ = 0.7,
    c = 6.0,
    σ = 2.0,
    w = w,
    p = p)

V, U, w_bar = solve_mccall_model(mcm)
U_vec = U .* ones(mcm.w)

plot(mcm.w, 
     [V U_vec],
     lw = 2, 
     α = 0.7, 
     label=["V" "U"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 45.60 
 
 
 45.65 
 
 
 45.70 
 
 
 45.75 
 
 
 
 
 
 
 
 V 
 
 
 
 U

In [3]:
c_vals = linspace(2, 12, 25)  
w_bar_vals = similar(c_vals)

for (i, c) in enumerate(c_vals)
    mcm = @NT( #Can drop @NT with Julia v0.7/v1.0
        u = u,
        α = 0.2,
        β = 0.98,
        γ = 0.7,
        c = c,
        σ = 2.0,
        w = w,
        p = p)
    V, U, w_bar = solve_mccall_model(mcm)
    w_bar_vals[i] = w_bar
end

plot(c_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="unemployment compensation",
     ylabel="reservation wage",
     label="w bar as a function of c")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 12 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 14 
 
 
 15 
 
 
 unemployment compensation 
 
 
 reservation wage 
 
<polyline clip-path="url(#clip5302)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 44.6753,360.065 66.9936,360.065 89.312,360.065 111.63,360.065 133.949,360.065 156.267,360.065 178.585,360.065 200.904,325.24 223.222,290.415 245.54,255.589 
 267.859,255.589 290.177,220.764 312.495,185.938 334.813,185.938 357.132,151.113 379.45,151.113 401.768,116.287 424.087,116.287 446.405,81.4619 468.723,81.4619 
 491.042,81.4619 513.36,46.6365 535.678,46.6365 557.997,46.6365 580.315,11.811 
 "/>
 
 
 
 
 w bar as a function of c

In [4]:
γ_vals = linspace(0.05, 0.95, 25)  
w_bar_vals = similar(γ_vals)

for (i, γ) in enumerate(γ_vals)
    mcm = @NT( #Can drop @NT with Julia v0.7/v1.0
        u = u,
        α = 0.2,
        β = 0.98,
        γ = γ,
        c = 6.0,
        σ = 2.0,
        w = w,
        p = p)
    V, U, w_bar = solve_mccall_model(mcm)
    w_bar_vals[i] = w_bar
end


plot(γ_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="job offer rate",
     ylabel="reservation wage",
     label="w bar as a function of \\gamma")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 10.0 
 
 
 10.5 
 
 
 11.0 
 
 
 11.5 
 
 
 12.0 
 
 
 12.5 
 
 
 job offer rate 
 
 
 reservation wage 
 
<polyline clip-path="url(#clip5502)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 54.5598,360.065 76.4662,360.065 98.3727,360.065 120.279,360.065 142.186,360.065 164.092,360.065 185.999,360.065 207.905,360.065 229.811,360.065 251.718,360.065 
 273.624,290.415 295.531,290.415 317.437,220.764 339.344,220.764 361.25,220.764 383.157,151.113 405.063,151.113 426.97,151.113 448.876,151.113 470.783,81.4619 
 492.689,81.4619 514.596,81.4619 536.502,81.4619 558.408,81.4619 580.315,11.811 
 "/>
 
 
 
 
 w bar as a function of 
 
 
 γ